In [1]:
# importing libraries
import pandas as pd
import sqlite3 as sql

In [2]:
con = sql.connect('sql-murder-mystery.db')
con

In [3]:
#looking for data for the crime scene report.
query_1 = '''
select * from crime_scene_report
where city = "SQL City" order by date;
'''
# setting the dataframe width to maximum
pd.set_option('display.max_colwidth',None)
# running the query
pd.read_sql_query(query_1,con)

,date,type,description,city
0,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
1,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
2,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City
3,20180103,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual.",SQL City
4,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
5,20180115,assault,Report Not Found,SQL City
6,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
7,20180215,murder,REDACTED REDACTED REDACTED,SQL City
8,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


 *Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".*

 Got our crime scene report, as per the report there are 2 witnesses.

In [4]:
# checking personal details of both witnesses.
# First Witness
query_2 = '''
select * from person where address_street_name = "Northwestern Dr"
order by address_number desc;
'''
pd.read_sql_query(query_2,con)[:1]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [5]:
# Second Witness
query_3 = '''
select * from person where name like "%Annabel%" and address_street_name = "Franklin Ave";
'''
pd.read_sql_query(query_3,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [6]:
# view the interview of both the witnesses taken after the murder.
query_4 = '''
select * from interview where person_id = "14887" or person_id = "16371";
'''
pd.read_sql_query(query_4,con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


So, there are 2 clues-

* Killer is a man and he is a member of gym with the status of Gold Membership and having the membership number starting with "48Z" and left in a car with a number plate "H42W".
* On 9th January he was working out in the gym.

In [7]:
# Checking the gym database with the above details.
query_5 = '''
select * from get_fit_now_check_in where membership_id like "%48Z%" and check_in_date = "20180109"
order by check_in_date;
'''
pd.read_sql_query(query_5,con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


* Two members found and their membership id is (48Z7A & 48Z55).

In [8]:
# now check the car details
query_6 = '''
select * from drivers_license where plate_number like "%H42W%";
'''
pd.read_sql_query(query_6,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


* Two males and One female containing the number H42W.

In [9]:
# Checking personal details of both males.
query_7 = '''
select * from person where license_id = "423327" or license_id = "664760";
'''
pd.read_sql_query(query_7,con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [11]:
# check which of these two are a member of gym.
query_8 = '''
select * from get_fit_now_member where person_id = "51739" or person_id = "67318";
'''
pd.read_sql_query(query_8,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


**Finally, found the murderer - Jeremy Bowers** Both the membership id and status also matches as per the information we found earlier.

_Now to confirm, visit the website and run the query:-_

INSERT INTO solution VALUES (1, 'Jeremy Bowers');

SELECT value FROM solution;

**Result is :-**

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

In [12]:
# Theres more in this, reading the transcript of the murderer.
query_9 = '''
select * from interview where person_id = "67318";
'''
pd.read_sql_query(query_9,con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


_So the real villian is the woman with Tesla car and red hair. Using these clues find the real mastermind behind the murder._

In [13]:
query_10 = '''
select * from drivers_license where car_make = "Tesla" and car_model = "Model S" 
and gender = "female" and hair_color = "red";
'''
pd.read_sql_query(query_10,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


 _There are three women with Tesla Model S and red hair color._

In [14]:
# checking the personal detials of the three womens.
query_11 = '''
select * from person where license_id = "202298" or license_id = "291182" or license_id = "918773";
'''
pd.read_sql_query(query_11,con)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [19]:
#checking the event SQL symphony concert
query_12 = '''
select person_id, count(*), event_name from facebook_event_checkin group by person_id having count(*) =3
and event_name = "SQL Symphony Concert" and date like "%201712%";
'''
pd.read_sql_query(query_12, con)

,person_id,count(*),event_name
0,24556,3,SQL Symphony Concert
1,99716,3,SQL Symphony Concert


**Finally, found the real mastermind of this whole mystery- Miranda Priestly**

_Miranda Priestly matches all the conditions for the mastermind we found earlier_

* _She has red hair and has Tesla Model S as well as she also attended the SQL Symphony Concert_

 Now, confirming it on the website by running the query:-

INSERT INTO solution VALUES (1, 'Miranda Priestly');

SELECT value FROM solution;

**Result is :-**

_Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!_